In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(f"⚠️ 메모리 설정 실패: {e}")

In [ ]:
import tensorflow as tf

# GPU 사용
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Import

In [ ]:
!pip install tensorflow_hub

In [ ]:
!pip install transformers

In [ ]:
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Embedding, Dropout, Dense, Flatten, Concatenate, Multiply, Layer, Attention, GRU, Bidirectional, MultiHeadAttention, LayerNormalization, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.activations import relu, sigmoid
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras.backend as K
import tensorflow_hub as hub
import random
import time
import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error
import re, os

In [ ]:
def set_seed():
    seed_num = 42
    tf.random.set_seed(seed_num)
    np.random.seed(seed_num)
    random.seed(seed_num)

In [ ]:
from tensorflow.python.client import device_lib
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Data load

In [ ]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '/content/drive/MyDrive/FFRS/FFRS_Data')
DATA_FILE = 'LDA_Musical_Instruments.pkl'

In [ ]:
full_path = f'{DATA_DIR}/{DATA_FILE}'
amzn = pd.read_pickle(full_path)

In [ ]:
# Musical_Instruments 데이터 BERT 임베딩 값
M_BERT_TRAIN_FILE = 'musical_new_bert_output_train.npy'
M_BERT_TEST_FILE = 'musical_new_bert_output_test.npy'
Musical_Instruments_train_path = f'{DATA_DIR}/{M_BERT_TRAIN_FILE}'
Musical_Instruments_test_path = f'{DATA_DIR}/{M_BERT_TEST_FILE}'
amzn_train_input_text = np.load(Musical_Instruments_train_path)
amzn_test_input_text = np.load(Musical_Instruments_test_path)

In [ ]:
# ID 라벨링
amzn.reviewerID = amzn.reviewerID.astype('category')
amzn.asin = amzn.asin.astype('category')
amzn.loc[:, 'asin'] = amzn.asin.cat.codes
amzn.loc[:, 'reviewerID'] = amzn.reviewerID.cat.codes

# Data Split
amzn_train, amzn_test = train_test_split(amzn, test_size=0.2, shuffle=True, random_state=42)

# Rating Data
train_rating = amzn_train['overall'].to_numpy(dtype=np.float32)
test_rating = amzn_test['overall'].to_numpy(dtype=np.float32)

print("Users:", amzn.reviewerID.nunique(), "Items:", amzn.asin.nunique())

<ipython-input-12-3b1b65f3bc18>:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[    0     0     5 ... 60180 60181 60181]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  amzn.loc[:, 'asin'] = amzn.asin.cat.codes
<ipython-input-12-3b1b65f3bc18>:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[25368 14452 26277 ... 26377 34131  8307]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  amzn.loc[:, 'reviewerID'] = amzn.reviewerID.cat.codes


Users: 40670 Items: 60182


컬럼 필터링

In [ ]:
# Musical_Instruments dataset
## Training Dataset
train_user_id = np.array(amzn_train['reviewerID'])
train_item_id = np.array(amzn_train['asin'])
train_topic = np.array(amzn_train[['Topic 0', 'Topic 1', 'Topic 2', 'Topic 3', 'Topic 4', 'Topic 5']])

## Test Dataset
test_user_id = np.array(amzn_test['reviewerID'])
test_item_id = np.array(amzn_test['asin'])
test_topic = np.array(amzn_test[['Topic 0', 'Topic 1', 'Topic 2', 'Topic 3', 'Topic 4', 'Topic 5']])

## Model(RMSE, attention)

In [ ]:
# RMSE 함수 정의
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [ ]:
def Scaled_dot_product_attention(query, key, value, mask=None): # mask=True, None / True하면 성능 안좋음.
    matmul_qk = tf.matmul(query, key, transpose_b=True)
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)
    attention_weights = tf.nn.softmax(logits, axis=-1)
    output = tf.matmul(attention_weights, value)
    return output

In [ ]:
def point_wise_feed_forward_network(d_model, dff):
  return Sequential([
      layers.Dense(dff, activation='relu'),
      layers.Dense(d_model)
  ])

def multi_head_attention_block(feature_1, feature_2, num_heads, feature_dims, dff=256, dropout_rate=0.2):
    # Multi-Head Attention
    multi_head = MultiHeadAttention(num_heads=num_heads, key_dim=feature_dims)(feature_1, feature_2) #, return_attention_scores=True
    multi_head = Dropout(dropout_rate)(multi_head)
    multi_head = LayerNormalization(epsilon=1e-6)(feature_1 + multi_head)

    # Point-wise Feed Forward Network
    Feed_forward = point_wise_feed_forward_network(feature_dims, dff=dff)(multi_head)
    Feed_forward = Dropout(dropout_rate)(Feed_forward)
    Feed_forward = LayerNormalization(epsilon=1e-6)(multi_head + feature_2 + Feed_forward)

    # Flatten
    Feed_forward = Flatten()(Feed_forward)

    return Feed_forward

# HNNER 모델

In [ ]:
def ModelBuild(user_num, item_num, topic_num, num_heads, feature_dims, drop_rate):

    '''
    1. ID Embedding Process
    '''
    # User ID
    user_input = Input(shape=(1,), dtype='int32', name='User_Input')
    user_embedding = Embedding(user_num, feature_dims, name='User_Emb')(user_input)
    user_embedding = Flatten(name='User_Flatten')(user_embedding)

    # Item ID
    item_input = Input(shape=(1,), dtype='int32', name='Item_Input')
    item_embedding = Embedding(item_num, feature_dims, name='Item_Emb')(item_input)
    item_embedding = Flatten(name='Item_Flatten')(item_embedding)

    #ID Concate
    user_item = Concatenate()([user_embedding, item_embedding])

    # ID MLP
    mlp_user_item = Dense(units=64, activation='relu', name='ID_MLP1')(user_item)

    '''
    2. Deep Feature Process
    '''
    # Deep Feature(BERT+Bi-GRU)
    # BERT
    bert_input = Input(shape=(768, ), dtype='float32', name='bert_input')
    bert_input = Dropout(drop_rate)(bert_input)
    bert_ex = tf.expand_dims(bert_input, axis=1)
    # Bi-GRU
    deep_feature  = Bidirectional(GRU(256), name='Bi_GRU')(bert_ex)
    # Self_attention
    self_attention_deep = Scaled_dot_product_attention(deep_feature, deep_feature, deep_feature)
    self_attention_deep = Dropout(drop_rate)(self_attention_deep)

    '''
    3. Hand Feature Process
    '''
    # Hand Feature(LDA)
    input_topic = Input(shape=(topic_num, ), dtype='float32', name='Topic_Input')

    topic_MLP = Dense(units=16, activation='relu', name='topic_MLP_0')(input_topic)
    topic_MLP = Dropout(drop_rate, name='Tpic_Drop_0')(topic_MLP) #
    topic_MLP = Dense(units=32, activation='relu', name='topic_MLP_1')(topic_MLP)
    topic_MLP = Dropout(drop_rate, name='Tpic_Drop_1')(topic_MLP)
    topic_MLP = Dense(units=64, activation='relu', name='topic_MLP_2')(topic_MLP)
    topic_MLP = Dropout(drop_rate, name='Tpic_Drop_2')(topic_MLP)
    topic_MLP = Dense(units=128, activation='relu', name='topic_MLP_3')(topic_MLP)
    topic_MLP = Dropout(drop_rate, name='Tpic_Drop_3')(topic_MLP)
    topic_MLP = Dense(units=256, activation='relu', name='topic_MLP_4')(topic_MLP)
    topic_MLP = Dropout(drop_rate, name='Tpic_Drop_4')(topic_MLP)
    topic_MLP = Dense(units=512, activation='relu', name='topic_MLP_5')(topic_MLP)
    topic_MLP = Dropout(drop_rate, name='Tpic_Drop_5')(topic_MLP)
    # Self_attention
    self_attention_hand = Scaled_dot_product_attention(topic_MLP, topic_MLP, topic_MLP)
    self_attention_hand = Dropout(drop_rate)(self_attention_hand)


    '''
    4. Deep-Hand Fusion Process
    '''
    # Multi-Head Attention
    deep_feature_MLP = tf.expand_dims(deep_feature, axis=1)
    hand_feature = tf.expand_dims(topic_MLP, axis=1)
    # Deep Feature
    forward_deep = multi_head_attention_block(deep_feature_MLP, hand_feature, num_heads=num_heads, feature_dims=512, dff=2048, dropout_rate=0.2)
    # Hand Feature
    forward_hand = multi_head_attention_block(hand_feature, deep_feature_MLP, num_heads=num_heads, feature_dims=512, dff=2048, dropout_rate=0.2)
    co_attention = Multiply()([forward_hand, forward_deep])
    text_all = Concatenate()([co_attention, self_attention_deep, self_attention_hand])

    '''
    Rating Prediction Process
    '''
    overall = Concatenate()([mlp_user_item, text_all])
    mlp_overall = Dense(units=512, activation='relu', name='Prediction_Layer_0')(overall)
    mlp_overall = Dropout(drop_rate, name='Prediction_Drop_0')(mlp_overall)
    mlp_overall = Dense(units=256, activation='relu', name='Prediction_Layer_1')(mlp_overall)
    mlp_overall = Dropout(drop_rate, name='Prediction_Drop_1')(mlp_overall)
    mlp_overall = Dense(units=128, activation='relu', name='Prediction_Layer_2')(mlp_overall)
    mlp_overall = Dropout(drop_rate, name='Prediction_Drop_2')(mlp_overall)
    mlp_overall = Dense(units=64, activation='relu', name='Prediction_Layer_3')(mlp_overall)
    mlp_overall = Dropout(drop_rate, name='Prediction_Drop_3')(mlp_overall)
    mlp_overall = Dense(units=32, activation='relu', name='Prediction_Layer_4')(mlp_overall)
    mlp_overall = Dropout(drop_rate, name='Prediction_Drop_4')(mlp_overall)
    mlp_overall = Dense(units=16, activation='relu', name='Prediction_Layer_5')(mlp_overall)
    mlp_overall = Dropout(drop_rate, name='Prediction_Drop_5')(mlp_overall)
    mlp_overall = Dense(units=8, activation='relu', name='Prediction_Layer_6')(mlp_overall)
    mlp_overall = Dropout(drop_rate, name='Prediction_Drop_6')(mlp_overall)
    output = Dense(units=1, dtype='float32', activation='relu', name='Output')(mlp_overall)
    model = Model(inputs=[user_input, item_input, bert_input, input_topic], outputs=output)
    return model

In [ ]:
user_num = amzn.reviewerID.nunique() #63931
item_num = amzn.asin.nunique() #47243
feature_dims = 64
topic_num = 6
num_heads = 2
drop_rate = 0.1
adam = Adam(learning_rate= 0.001)
proposed_model = ModelBuild(user_num, item_num, topic_num, num_heads, feature_dims, drop_rate)

In [ ]:
proposed_model.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_absolute_error', 'mean_squared_error', rmse])
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5, restore_best_weights=True)

In [ ]:
with tf.device('/device:GPU:0'):
    result = proposed_model.fit([train_user_id, train_item_id, amzn_train_input_text, train_topic], train_rating, batch_size=128, epochs=100, validation_split=1/8, callbacks=[es])

Epoch 1/100
1967/1967 [==============================] - 58s 21ms/step - loss: 1.6383 - mean_absolute_error: 0.9712 - mean_squared_error: 1.6383 - rmse: 1.2596 - val_loss: 0.8122 - val_mean_absolute_error: 0.7165 - val_mean_squared_error: 0.8122 - val_rmse: 0.8974
Epoch 2/100
1967/1967 [==============================] - 38s 19ms/step - loss: 0.9087 - mean_absolute_error: 0.7046 - mean_squared_error: 0.9087 - rmse: 0.9496 - val_loss: 0.7219 - val_mean_absolute_error: 0.6410 - val_mean_squared_error: 0.7219 - val_rmse: 0.8457
Epoch 3/100
1967/1967 [==============================] - 38s 19ms/step - loss: 0.6260 - mean_absolute_error: 0.5681 - mean_squared_error: 0.6260 - rmse: 0.7879 - val_loss: 0.7204 - val_mean_absolute_error: 0.5686 - val_mean_squared_error: 0.7204 - val_rmse: 0.8436
Epoch 4/100
1967/1967 [==============================] - 38s 19ms/step - loss: 0.4711 - mean_absolute_error: 0.4816 - mean_squared_error: 0.4711 - rmse: 0.6826 - val_loss: 0.7207 - val_mean_absolute_error:

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
prediction = proposed_model.predict([test_user_id, test_item_id, amzn_test_input_text, test_topic])
MAE_temp = mean_absolute_error(test_rating, prediction)
RMSE_temp = mean_squared_error(test_rating, prediction, squared = False)

print(f'MAE: {MAE_temp:.4f}')
print(f'RMSE: {RMSE_temp:.4f}')

2248/2248 [==============================] - 12s 5ms/step
MAE: 0.5712
RMSE: 0.8565
